## MeshConfHelper Tutorial

This notebook goal is to help you understand how conf.json and \<categorie\>.json work

This is an exemple of a simple conf.json

```json
{
  "directed": true,
  "multigraph": true,
  "graph": {
    "name": "Chain3mesh"    <=   This will be display at url "/" in the select field
  },
  "nodes": [
    {
      "id": "node1",    <=   Primary Key of the node, must be unique for each node
      "label": "Node1",    <=   Label displayed in url "/<graph.name>"
      "folder": "node1",    <=   The folder name in folder "data/Chain3mesh/node1" or "data/<graph.name>/<folder>
      "position": [0, 0],    <=   Grid position in the builder
      "permissions": ["select", "list"],    <=   Permission and forms displayed
      "files": ["node.json", "another.json"]    <=   conf files within folder (those files could be shared easily)
    },
    {
      "id": "node2",
      "label": "node2",
      "folder": "node2",
      "position": [0, 1],
      "permissions": ["select", "list", "rotate"],
      "files": ["node.json"]
    },
    {
      "id": "node3",
      "label": "node3",
      "folder": "node3",
      "position": [0, 2],
      "dextral": "right",    <=   when dextral is set to something, the node handle mirroring different dextral stl conf in <categorie>.json (boolean True False) (check ** bellow)
      "permissions": ["select", "list", "rotate"],
      "files": ["node.json"]
    }
  ],
  "links": [
    {"source": "node1", "target": "node2"},    <=   nodes links to create a Directed acyclic graph
    {"source": "node2", "target": "node3", "merge": true},    <=   merge means Mesh node2 and node3 will be merged in the downloaded zip
  ]
}
```

### This is an exemple of a simple node.json configuration of Nodes ! (located in data/categorie)
The arborescence will be :
```
data/
    Chain3mesh/
        categorie_1/
            3DprintOriceps.stl
        categorie_2/
            file_name.stl
```

vertex, facet and vertex_list values are dummy data in this exemple

```json
{
    "categorie_1": {    <=   category of mesh files, display in selectField
        "desc": {
            "display": "Categorie 1",    <=   Unused Yet, should be Label displayed
            "path": "data/Chain3mesh/categorie_1/"    <=   path to the files bellow # might be refactored later.
        },
        "stl": {
            "Oriceps": {    <=   label to display in listField   
                "file": "skull-armature.stl",    <=   file name in www.thingiverse.com/thing:5035669/files to download    EASY TO ADD
                "thingiverse": 5035669,    <=   id of the thingiverse thing : www.thingiverse.com/thing:5035669/files    EASY TO ADD
                "node1": {
                    "vertex": 2966    <=   Index of the vertex used for nodes links in Trimesh     THIS IS THE BORING PART WHERE COMMUNITY CAN HELP
                },
                "node2": {
                    "facet": 37550    <=   Index of the facet used for nodes links in Trimesh     THIS IS THE BORING PART WHERE COMMUNITY CAN HELP
                },
                "node3": {
                    "left": {    <=   Dextral value used to check if the mesh need to be mirrored (check ** in previous cell)
                        "facet": 5851
                    },
                    "right": {
                        "facet": 7510
                    }
                },
                "node4": {
                    "vertex_list":  [12, 3983]    <=   Index of multiple vertex used in Trimesh, a centroid vertex is generated for nodes links    THIS IS THE BORING PART WHERE COMMUNITY CAN HELP
                },
                "coef_merge": 0.1    <=   coefficient to let two linked meshes more or less in superposition
                "rotate": 15    <=   rotation to apply between two nodes (target is rotated), combined with rotate permission in conf.json
            },
            ...
        }
    },
    "categorie_2": {
        "desc": {
            "display": "Categorie 2",
            "path": "data/Chain3mesh/categorie_2/"
        },
        "stl": {
            "Other Mesh Label": {
                "file": "file_name.stl",
                "node1": {
                    "facet": 12
                },
                "node2": {
                    "vertex": 32
                },
                "node3": {
                    "left": {
                        "vertex_list": [12, 3983] 
                    },
                    "right": {
                        "vertex": 12
                    }
                },
                "node4": {
                    "vertex": 12
                }
            }
        }
    }
}
```

# How to find facet ? 

In [10]:
from utils import get_faces
from trimesh import load

In [11]:
mesh = load("skull-armature.stl") # Path to tour stl file

face_normals all zero, ignoring!


In [12]:
mesh.show() 

In [5]:
get_faces(mesh, min_=150, max_=155)  # play with min_ and max_ value to find the right facet, use real size logic !

vertex index is: 5 facet area is 154.04548910274528mm
{"facet": 5}


### check under the skull, the face is now green, if it's the right one

In [6]:
mesh.show()

# FIND VERTEX AND VERTEX LIST

### For this part we will need blender https://www.blender.org/download/

![blender](blender.png)

DON'T MOVE THE FILE OR EDIT HIM !

Red : 
1. go in edit mode
2. unselect all vertices
3. select one vertice/ multiple vertices where you want to link the file with another one (multiple vertices selected will be averaged to get the center, keep shift to select multiple vertices)

Green : 
1. go in python console mode, follow click 1, 2, 3, 4
2. type those lines in blender console :

```python
#for blender to get current selection
m = [i.index for i in bmesh.from_edit_mesh(bpy.context.active_object.data).verts if i.select]
vertices = [ver for ver in bpy.context.active_object.data.vertices.values() if ver.index in m]
print([(x.co[0], x.co[1], x.co[2]) for x in vertices])
```

3. copy past the output result in blender as bellow

In [1]:
from utils import find_vertices
from trimesh import load
mesh = load("skull-armature.stl") # Path to tour stl file

face_normals all zero, ignoring!


copy past the printed value as bellow

In [7]:
#vertices_to_find = <copy past here>
#find_vertices(mesh, *vertices_to_find)

In [4]:
vertices_to_find = [[0.3492145240306854, -18.59728240966797, 21.122314453125], [-0.003770826617255807, -18.607858657836914, 21.1286563873291]]
find_vertices(mesh, *vertices_to_find)

92
4667
{"vertex_list": [92, 4667]}


![blender_nose](blender2.png)

In [8]:
vertice_ton_find = [[0.1860094666481018, 2.6514551639556885, 44.99243927001953]]
find_vertices(mesh, *vertice_ton_find)

4808
{"vertex": 4808}


The stl file configuration in \<categorie>.json will look like this :
```json
"Other Mesh Label": {
    "file": "skull-armature.stl",
    "thingiverse": 5035669,
    "body": {
        "facet": 5
    },
    "nose": {
        "vertex_list": [92, 4667]
    },
    "hat": {
        "vertex": 4808
    }
}
```


## you can add it and share it to the world, thank you for contributing